In [ ]:
#!pip install autotrain-advanced
#!pip install huggingface_hub

In [ ]:
#!autotrain setup --update-torch

In [ ]:
#connect to google drive
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
#!autotrain llm \
#--train \
#--project_name "llama-2-7B" \
#--model "TinyPixel/Llama-2-7B-bf16-sharded" \
#--data_path /content/drive/MyDrive/Sample \
#--text_column text \
#--use_peft \
#--use_int4 \
#--learning_rate 2e-4 \
#--train_batch_size 9 \
#--num_train_epochs 11 \
#--trainer sft > training.log &

In [ ]:
from transformers import LlamaModel, AutoModelForCausalLM, AutoTokenizer
from torch.nn import DataParallel
import time
import torch
import gc

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("llama-2-7B")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/llama-2-7B").to('cuda:0')

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
#%cp -r /content/llama-2-7B /content/drive/MyDrive/llama\ model

In [ ]:
import pandas as pd

In [ ]:

df = pd.read_csv("qt.csv")
df.shape
df.head(5)

,Unnamed: 0,subreddit,commentId,commentTimeStamp,commentAuthor,comment,parseedComment,topic,commentCategory,text,generated_text
0,19406.0,CredibleDefense,ie5kfoe,1.656483e+09,Duncan-M,That's not at all what I wrote. Long term pros...,That's not at all what I wrote. Long term pros...,ukraine Conflict,Other,"You are a war comment analyst, access the stan...","###Human:\nYou are a war comment analyst, acce..."
1,25894.0,worldnews,k738b7h,1.698675e+09,razorxent,https://www.dazeddigital.com/politics/article/...,https://www.dazeddigital.com/politics/article/...,Israel,Other,"You are a war comment analyst, access the stan...","###Human:\nYou are a war comment analyst, acce..."
2,41367.0,news,gvh12e9,1.619115e+09,nine_legged_stool,I'll be honest. I'm Ukrainian-born and I could...,I'll be honest. I'm Ukrainian-born and I could...,.russia.ukraine.,Other,"You are a war comment analyst, access the stan...","###Human:\nYou are a war comment analyst, acce..."
3,27800.0,FlightTracker,hx7rnn1,1.645042e+09,FlightTrackerBot,\nAuthor: https://www.reddit.com/user/Cambridg...,\nAuthor: https://www.reddit.com/user/Cambridg...,ukraine Conflict,Other,"You are a war comment analyst, access the stan...","###Human:\nYou are a war comment analyst, acce..."
4,48500.0,worldnews,i097mka,1.647015e+09,justsigndupforthis,There seems to be a plan for a merger\n\nhttps...,There seems to be a plan for a merger\n\nhttps...,ukraine,Other,"You are a war comment analyst, access the stan...","###Human:\nYou are a war comment analyst, acce..."


In [ ]:
stride = 4
start = time.time()

def elapsed_time():
      elapsed = (time.time() - start)
      s = int(elapsed % 60)
      m = int(((elapsed - s) // 60) % 60)
      h = int((elapsed - m * 60 - s) // 3600)
      return f"[{h:02}h {m:02}m {s:02}s]"

for i in range(1728, df.shape[0], stride):
  texts = df.text.iloc[i:i+stride].apply(
      lambda x: f"###Human:\n{x}\n\n###Assistant:\n").to_list()

  input_ids = tokenizer(texts, return_tensors="pt", padding=True)\
      .to(model.device)

  output = model.generate(**input_ids,
                          max_length=1536,
                          num_return_sequences=1)

  del input_ids
  time.sleep(3)
  gc.collect()
  torch.cuda.empty_cache()
  time.sleep(3)

  for j in range(stride):
    if i + j >= df.shape[0]:
      break

    df.generated_text.iloc[i+j] = tokenizer.decode(
        output[j], skip_special_tokens=True)

  del output
  gc.collect()
  torch.cuda.empty_cache()
  time.sleep(2)
  df.to_csv(f"qt.csv",index=None)
  print(f"[{elapsed_time()}] Processed: upto {i+stride}")

<ipython-input-11-1f67f72757bd>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.generated_text.iloc[i+j] = tokenizer.decode(


[[00h 00m 19s]] Processed: upto 1732
[[00h 00m 35s]] Processed: upto 1736
[[00h 00m 54s]] Processed: upto 1740
[[00h 01m 10s]] Processed: upto 1744
[[00h 01m 26s]] Processed: upto 1748
[[00h 01m 42s]] Processed: upto 1752
[[00h 01m 58s]] Processed: upto 1756
[[00h 02m 14s]] Processed: upto 1760
[[00h 02m 31s]] Processed: upto 1764


Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


[[00h 02m 49s]] Processed: upto 1768
[[00h 03m 10s]] Processed: upto 1772
[[00h 03m 27s]] Processed: upto 1776
[[00h 03m 43s]] Processed: upto 1780
[[00h 04m 00s]] Processed: upto 1784
[[00h 04m 17s]] Processed: upto 1788
[[00h 04m 33s]] Processed: upto 1792
[[00h 04m 51s]] Processed: upto 1796
[[00h 05m 10s]] Processed: upto 1800
[[00h 05m 26s]] Processed: upto 1804
[[00h 05m 41s]] Processed: upto 1808
[[00h 05m 58s]] Processed: upto 1812
[[00h 06m 14s]] Processed: upto 1816
[[00h 06m 31s]] Processed: upto 1820
[[00h 06m 48s]] Processed: upto 1824
[[00h 07m 04s]] Processed: upto 1828
[[00h 07m 19s]] Processed: upto 1832
[[00h 07m 36s]] Processed: upto 1836
[[00h 07m 55s]] Processed: upto 1840
[[00h 08m 12s]] Processed: upto 1844
[[00h 08m 29s]] Processed: upto 1848
[[00h 08m 48s]] Processed: upto 1852
[[00h 09m 04s]] Processed: upto 1856
[[00h 09m 19s]] Processed: upto 1860
[[00h 09m 35s]] Processed: upto 1864
[[00h 09m 51s]] Processed: upto 1868
[[00h 10m 08s]] Processed: upto 1872
[